In [12]:
# Process to map patient address to SVI

import pyodbc
import pandas as pd
import os 
import sqlite3 
import shutil
from sqlalchemy import create_engine
import urllib
import requests

# SQLAlchemy
# Database connection details
server = 'PRD-APP-PBI-GW\\UCRHEALTH_PROD'  
database = 'ucr_health'  
username = 'ucr_svc_analytics'  
password = 'Ucr53Rv!c3'


# Construct the connection string
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
encoded_conn_str = urllib.parse.quote_plus(connection_string)

# Create the SQLAlchemy engine 
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={encoded_conn_str}')
query = '''
SELECT [pat_id]
      ,[mrn]
      ,[pat_name]
      ,[add_line_1] as address
      ,[city]
      ,s.ABBR as [state]
      ,[zip]
  FROM [ucr_health].[src].[patient] as p
  left outer join [ucr_health].[src].[zc_state] as s
  on p.STATE_C = s.STATE_C
'''
# Read query results directly into a DataFrame
df = pd.read_sql(query, engine)


# Close the connection (optional with SQLAlchemy, but good practice)
engine.dispose() 

In [14]:
df.columns

Index(['pat_id', 'mrn', 'pat_name', 'address', 'city', 'state', 'zip'], dtype='object')

In [16]:
def get_census_tract(address):
    # Base URL for the Geocoding Services API
    base_url = "https://geocoding.geo.census.gov/geocoder/geographies/address"

    # Parameters for the API request
    params = {
        "street": address['street'],
        "city": address.get('city', ''),
        "state": address.get('state', ''),
        "zip": address.get('zip', ''),
        "benchmark": "Public_AR_Current",
        "vintage": "Current_Current",
        "format": "json",
        "layers": "Census Tracts"
    }

    try:
        # Send the GET request
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise HTTPError for bad responses

        # Parse the JSON response
        data = response.json()

        # Extract the Census Tract information
        geographies = data.get("result", {}).get("geographies", {})
        census_tracts = geographies.get("Census Tracts", [])

        if census_tracts:
            tract = census_tracts[0].get("TRACT", "N/A")
            return tract
        else:
            return "No Census Tract found"

    except requests.exceptions.RequestException as e:
        return f"Error: {e}"

def process_addresses(df):
    # Ensure the input DataFrame has the required columns
    required_columns = {'address', 'city', 'state', 'zip'}
    if not required_columns.issubset(df.columns):
        raise ValueError(f"Input DataFrame must contain the following columns: {required_columns}")

    # Create a list to store the results
    results = []

    # Iterate through the DataFrame rows
    for _, row in df.iterrows():
        address = {
            "street": row['address'],
            "city": row['city'],
            "state": row['state'],
            "zip": row['zip']
        }
        tract = get_census_tract(address)
        results.append(tract)

    # Add the results to the DataFrame
    df['Census Tract'] = results
    return df



In [18]:
updated_df = process_addresses(df)

KeyboardInterrupt: 

In [20]:
updated_df.head()

NameError: name 'updated_df' is not defined